.. toctree::

# FAQ

## What is Steppy standard documentation framework?

The documentation framework is **Sphinx**

## What is the difference between scikit pipeline and steppy?

In a  pipeline a series of data-operations are connected together. In a data-analytics problem, this is usually a left-to-right series of calls of transformer calls ended by an estimator, that is a PIPELINE of invocations.

The major limitation the scikit learn Pipeline wrapper is that passed data object must be same input and output, i.e. data object(s) is(are) same and implicit thoughout Pipeline.

Thus step introduces two new wrappers, Step and Adapter to avoid these limitations:

Steps communicate data between each other with **Adapters**, which are implemented as Python dictionaries. This makes it possible to pass collections of arbitrary data types (Numpy arrays, Pandas dataframes, etc.). The basic structure is as follows:

    data_train = {'input':
                    {
                         'X': X_train,
                         'y': y_train,
                    }
                }

where X_train,y_train are local data objects, X,y are names of arguments to step-exec-object-instance and  ‘input'  is the name of the Adapter.


see https://github.com/neptune-ml/steppy-examples/blob/master/tutorials/1-getting-started.ipynb for more information.

## receiving an error "no module named deepsense"

In the repository folder do: 

    pip3 install -r requirements.txt 
    
this should solve this error.

## how to get around  xp =f(x)  to, x,xp = f(x) challenge

All of the argument/inputs of ALL step-exec-object-instance must be covered by the named Adapter, but all the named arguments of the named Adapter need not be used by the step-exec-object-instance. For example the following Adapter would work for a **Step** that needs only X .


    data_train = {'input':
                    {
            'X': X_train,
            'y': y_train,
            ‘z1’, some_other_bound_variable,
            ‘etc’, etc	
                    }
                }

## Are there other Resources for Steepy?

Yes. see  https://steppy.readthedocs.io

## Are there tutorials for Steepy?

Yes. In the form of notebooks. See https://github.com/neptune-ml/steppy-examples/tree/master/tutorials

## Is Steppy Threadsafe?

Steppy itself is  thread-safe. However, you will use Steppy with many other packages which may not be thread-safe. For example, numpy is thread-safe.
ndarrays can be accessed in a thread-safe manner, but you must be careful with state if you mutate an array.

In Pandas, deleting a column is usually not thread-safe as changing the size of a DataFrame usually results in a new Dataframe object. At some point this may change in Pandas and other python libaries as multi-cored CPUs are becoming common.

# Q&A from open-solution-data-science-bowl-2018

# Q&A from Home Credit Default Risk (Open souce solution for Kaggle)

August-2018

Dear Kagglers,

As you well know we share our work with the community for the benefit of all. We want to help those who are learning the ropes of data science get in the groove of things, we want to help those less organized make their code and process cleaner and finally we want those at the very top of the game to use our work (or parts of it) to develop state of the art solutions quicker and test the boundaries of what is possible for a given problem.
...

Let me just clarify that we work with Python3.5 and Ubuntu 16.04. We did not run any tests on other operating systems and we do not use Python3.6.

Also, please make sure that you are 100% compliant with our requirements.txt file. Differences in packages versions may lead to unexpected results :)

@starhao note that sometimes groupby operation yields empty array. Because of that, you are receiving warning you mentioned. It is ok, your experiment progress.

Finally, let me mention that end-to-end execution is something like 10-12h. Hence, indeed you may wait some time for the features to get computed. It happens here Step installment_payments_hand_crafted, fitting and transforming... 

(Ed: Also runs in approximately 3. hours in the following configuration:

    Model Name:	Mac Pro
    OS: High Sierra, version.10.6.13
    Processor Name:	12-Core Intel Xeon E5
    Processor Speed:	2.7 GHz
    Total Number of Cores:	12
    L2 Cache (per Core):	256 KB
    L3 Cache:	30 MB
    Memory:	64 GB
)

It heavily depends on your hardware. In this topic, some people mentioned their hardware configuration and time it took. It can be even hours, in case you have just few vCPUs.

:Authors: Kuba & Kamil

##  Is is possible to save features with your pipeline? 
I mean raw features, for train and test set, in the exact same shape as you feed them to your model?

:Author: narsil (kaggle handle)

### Answer

Steppy Step object is designed to handle stuff like this. There are 3 flags that are particularly important to your problem:

    persist_output = True
    cache_output = True
    load_persisted_output = False
    
In our case the Step that joins the features is called feature_joiner so go to this line and setup persist_output=True both during training and evaluation.

Then you need to generate the features. I would suggest you do it in one go with something like this:

    neptune run --config configs/neptune.yaml main.py train --pipeline_name lightGBM

it will dump your features in the /YOUR_EXPERIMENT/outputs directory.

Everything can be loaded with 

    sklearn.externals.joblib.load(filepath)

The same thing can be done for the test set. Just run

    neptune run --config configs/neptune.yaml main.py predict --pipeline_name lightGBM

Now with our code we divide train/valid by default so if you want to have the entire train features and not train/valid you need to go to the pipeline_manager and change

    train_data = {'application': {'X': train_data_split.drop(cfg.TARGET_COLUMNS, axis=1),
                                  'y': train_data_split[cfg.TARGET_COLUMNS].values.reshape(-1),
                                  'X_valid': valid_data_split.drop(cfg.TARGET_COLUMNS, axis=1),
                                  'y_valid': valid_data_split[cfg.TARGET_COLUMNS].values.reshape(-1)
                                  },
to this

    train_data = {'application': {'X': tables.application_train.drop(cfg.TARGET_COLUMNS, axis=1),
                                  'y': tables.application_train[cfg.TARGET_COLUMNS].values.reshape(-1),
                                  'X_valid':None,
                                  'y_valid': None
                                  },
                                  
It will most likely fail at training (because we need X_valid, y_valid) but is should generate the features correctly. If you have any trouble please let me know.


:Author: Jakub Czakon

##  how do I convert a saved feature file into a dataframe

:Author: incarnation (Kaggle handle)

### Answer

You can get data frame with all features by following code:

    from sklearn.externals import joblib
    df = joblib.load('path/to/your/feature_joiner')['features']

:Author: Miłosz Michta


##  Is the script in your repo by default use single cpu for model training? If yes, where to change the setting in your project to use multiple cpus?

Also,for example, the machine has 16 cpus, and if I want to use 8 cpu for the lgb modeling (I'd like to reserve the other 8 cpus for other tasks), how should I configure the setting in the project? Would setting "num_works=16", and "lgbm__nthread: 8" (in lgb parameters) be the correct configuration in such a case?


:Author: Shize Su (kaggle handle)

### Answer

Have you tried to change num_workers in neptune.yaml? By default it's setted to 1.

Also, If you want to run your training on 8 cpus, set num_workers: 8. The amount of your total cpu number doesn't matter.

(Ed: One cavet, is that most of the multi-cpu/core architectures, support 2 or more threads per cpu/core.  num_workers: should be set to thread count.  For example, Intel Core i7-7700K has 4 cores and 8 parellel threads.)

:Author: Miłosz Michta

## Where could we set (/change) the random seed for the kfold split? I didn't find random seed setting in the neptune.yaml file.

 :Author: Shize Su (kaggle handle)

### Answer

There is only one global random seed in pipeline_config.py: 

    RANDOM_SEED = 90210
    
To change random seed for only k-fold split, you can do this manually in pipeline_manager.py in _get_fold_generator function.

:Author: Miłosz Michta

## I change some lgb parameters in the neptune.yaml file and rerun the code. 

Then a weird error in model training raised up as follows:

    2018-08-21 17-13-18 home-credit >>> LightGBM, train data shape (48744, 1174)

    2018-08-21 17-13-18 home-credit >>> LightGBM, validation data shape (61503, 1174)

    2018-08-21 17-13-18 home-credit >>> LightGBM, train labels shape (246008,)

    2018-08-21 17-13-18 home-credit >>> LightGBM, validation labels shape (61503,)

    [LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN

    [LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN

And obviously the train data and train label shape doesn't match and error raised up when start model training. It seems that the code read the saved test feature data as train feature data which caused such error when rerunning the code. But I don't know why this could happen. Is that by default the code saved test feature data with the same file name as the train feature data and override it? To avoid such errors in rerunning the code (with different lgb parameters), are there anywhere in the project setting that I need to change?

Note that I run the code exactly as it is, except that I follow Jakub's suggestion below to setup persist_output=True in the feature_joiner before using the following command to run the code:

    python main.py -- train_evaluate_predict_cv --pipeline_name lightGBM

Thanks in advance!

Update: I checked the saved feature data files and verified that the saved train feature data files are indeed override by the saved test feature data files (they have the same file name "feature_joiner_fold_x"). @Milosz, would you mind letting me know which setting in the project code I should change to avoid such feature files overriding issue? Thanks!

:Author: incarnation (Kaggle handle)


### Answer


There are 3 flags in the Step constructor that you need to consider:

persist_output: True
load_persisted_output: True
cache_output: True
If you don't want to ovewrite anything during eval/test just make sure to set perist_output: False . All the steps used are defined in the pipeline_blocks.py . More specifically you need to take care of the feature_joiner .

It is also important to know that you can have a seperate experiment_dir for train and test so that you could just load your transformed features but change the classifier on top of it. In that case you would need to copy the experiment_dir/transformers to a new experiment directory. Also make sure to specify 'clean_experiment_directory_before_training: 0 in the neptune.yaml otherwise it will just remove everything from the experiment_dir . One last piece of the puzzle if you want to be steppy master is that if you have your lgbm trained in that folder steppy will simply load that model and transform it unless you pass force_fitting: True to the Step contractor for that step. Which means that running grid search or random search or simply retraining with different hyperparams you don't have to remove that lgbm transformer from experiment_dir/transformers but you can simply pass that flag and overwrite it.

:Author: Jakub Czakon


## I wanted to try some sklearn models and did not get it running. 

    python main.py -- train_evaluate_predict --pipeline_name random_forest

it will give me an error:

    pipeline = PIPELINES[pipeline_name](config=cfg.SOLUTION_CONFIG, train_mode=True) TypeError: 'dict' object is not callable

For me, this seems to be quite natural, as the sklearn models are defined as dictionaries with keys "train" and "inference" in the pipelines.py as opposed to the lgbm model.

Any tips would be much appreciated!

:Author: Maximilian Hahn


### Answer

You're right, to solve this you can simply change that:

    'random_forest': {'train': partial(sklearn_main,
                                            ClassifierClass=RandomForestClassifier,
                                            clf_name='random_forest',
                                            train_mode=True),
                     'inference': partial(sklearn_main,
                                                ClassifierClass=RandomForestClassifier,
                                                clf_name='random_forest',
                                                train_mode=False)
                    }
to this:

    'random_forest': partial(sklearn_main,
                         ClassifierClass=RandomForestClassifier,
                         clf_name='random_forest'),

:Author: Miłosz Michta

## is there a way to run it using Jupyter?

:Author: Daniel Burrueco

### Answer

If you are doing it in python.

    !python main.py -- train_evaluate_predict_cv --pipeline_name lightGBM

:Author: William Green

### Answer

What is being executed are pipelines from main.py. In practice in means that you can create notebook in the repository root, import required libs and execute one of these pipelines.

For example training pipeline is defined in the pipeline_manager.py file. Just make sure that you put correct paths to data.

:Author: Kamil (Kaggle handle)

##  I run out of disk space. Is it possible to run with the cache turned off?

:Author: Dromosys (Kaggle handle)

### Answer

Set 

    cache_output=False
    persist_output=False 
    load_persisted_output=False 
    
in pipelines.py and pipeline_blocks.py

:Author: Jakub Czakon (Kaggle handle)

##  It looks like the script was designed to run on neptune only. I don't see the difference between Fast Track and Step by step installation guide.

:Author: nlgn (Kaggle handle)

### Answer
Our code in neptune-agnostic, thus you can run it as Python script:

    python main.py train_evaluate_predict --pipeline_name lightGBM.

Full list of pipelines is here: lightGBM, XGBoost, random_forest, log_reg, svc. I'm still playing with XGBoost.

My intention for Fast Track was to give three short points for User who can clone repo, install reqs, etc.


:Author: kamil (Kaggle handle)

## Could you point me the right class structure and method names where to find save and load fold data?

### Answer

If you want to simply read them in go:

    from sklearn.externals import joblib
    feature_dict = joblib.load('PATH/TO/FEATURE_JOINER/')
    fetures = feature_dict['features']

:Author: Jakub Czakon (Kaggle handle)

## Is it possible to save target values via feature_joiner feature_joiner_valid while using cv? Or How can access cv target values and store?

:Author: kkaczmarek

### Answer

You can adjust FeatureJoiner to do that:

    class FeatureJoiner(BaseTransformer):
    def __init__(self, use_nan_count=False, **kwargs):
        super().__init__()
        self.use_nan_count = use_nan_count

    def transform(self, numerical_feature_list, categorical_feature_list, targets, **kwargs):
        features = numerical_feature_list + categorical_feature_list
        for feature in features:
            feature.reset_index(drop=True, inplace=True)
        features = pd.concat(features, axis=1).astype(np.float32)
        if self.use_nan_count:
            features['nan_count'] = features.isnull().sum(axis=1)

        outputs = dict()
        outputs['features'] = features
        outputs['feature_names'] = list(features.columns)
        outputs['categorical_features'] = self._get_feature_names(categorical_feature_list)
        outputs['targets'] = targets
        return outputs

    def _get_feature_names(self, dataframes):
        feature_names = []
        for dataframe in dataframes:
            try:
                feature_names.extend(list(dataframe.columns))
            except Exception as e:
                print(e)
                feature_names.append(dataframe.name)

        return feature_names
        
Remember to change the pipeline_blocks accordingly.

If you want it just for the adhoc purposes however I would simply dump the targets with _foldX suffix through pipeline_manager.py . For example:

    for fold_id, (train_idx, valid_idx) in enumerate(fold_generator):
    (train_data_split,
     valid_data_split) = tables.application_train.iloc[train_idx], tables.application_train.iloc[valid_idx]

     joblib.dump(train_data_split[cfg.TARGET_COLUMNS], 'train_target_fold_{}'.format(fold_id))

:Author: Jakub Czakon (Kaggle handle)

##  I am wondering: after running the code, where can I find the output, I mean, 'submission.csv'?

:Author:  DKADKA

### Answer

It is placed in the experiment_directory that you specified in the yaml file.

:Author: Kamil (Kaggle handle)

##  When I run any of the functions that include parallel apply the function gets hung up and does not move?

:Author: benedic2 (Kaggle handle)

### Answer

I observed this effect for other versions of Pandas:

1. make sure that you have version listed in the requirements file.
1. Check CPU and memory utilization. Very likely everything is just fine. It just takes some time to extract features from files.
1. If you are sure that you have issues with multiprocessing you can roll back to the standard Pandas. In such case you do not use parallel apply function. You simply change it to the operation on Pandas' group object. So something like groupobject.apply(func).reset_index(). This will do the same but with Pandas.

:Author: kamil (Kaggle handle)

## Is the number of estimators set anywhere for the LGBM models?

:Author: benedic2

### Answer

Yes, we set it in the configuration files. Look for lgbm__number_boosting_rounds. 

:Author: kamil (Kaggle handle)

## I saw you were using SHAP values to evaluate feature importance. How do you go about removing unimportant features? Do you remove simply remove all features with SHAP values below a certain threshold??

:Author: IINewton

### Answer

Actually, we are not removing any features. This could be really problematic. Take a look, that in our notebook we analyze data/model from fold_0 and features which have zero importance in this fold also have nonzero importance in others.

:Author: Miłosz Michta (Kaggle handle)

## from toolkit.sklearn_transformers.models import SklearnClassifier ModuleNotFoundError: No module named 'toolkit'

:Author: Omid Safarzadeh (Kaggle handle)

### Answer

I think that you did not install steppy-toolkit==0.1.5. try this: 

    pip3 install steppy-toolkit==0.1.5.

:Author: kamil (Kaggle handle)

## Is the command same for training after making adjustments to the parameters? Or, can I just run: -- train_evaluate_predict_cv command ? Just wondering if I have to go through the pipeline again.?

:Author: William Green (Kaggle handle)

### Answer
 
Remember to change the clean_experiment_directory in the neptune.yaml to False and force_fitting=True in the lightgbm Step. Having done that running it again will just load the features for each fold and train model on top of it.

:Author: Jakub Czakon

# Q&A from TGS Salt Identification Challenge  (Open souce solution for Kaggle)

## Goals

- establish solid benchmark for the competition,
- make this competition more approachable by giving starter code and providing help via discussion forum, promote the idea of clean and extensible code for Kaggle competitions :) 
- In this topic... you will read about open solution updates, new ideas, experiments and comments. Feel invited to participate in building this.

Have fun competing :) 

:Authors: Kuba & Kamil


## I got this error: File "open-solution-salt-identification-master/common_blocks/callbacks.py",

    line 150, in on_batch_end loss = loss.data.cpu().numpy()[0] IndexError: too many indices for array 
    
Could you give me some help? 
  
 :Author: tommao

### Answer

Please make sure that you have torch==0.3.1. On the more general level, please make sure that you have all requirements is place: requirements.txt.

:Author: Kamil (Kaggle handle)

##

### Answer

##

### Answer

##

# Q&A from open-solution-ship-detection
https://github.com/neptune-ml/open-solution-ship-detection


# Q&A from open-solution-value-prediction

# Q&A from open-solution-googleai-object-detection
https://github.com/neptune-ml/open-solution-googleai-object-detection

# Q&A from Mapping Challenge
https://www.crowdai.org/challenges/mapping-challenge


# Q&A from open-solution-value-prediction
https://github.com/neptune-ml/open-solution-value-prediction


# Q&A from open-solution-cdiscount-starter
https://github.com/neptune-ml/open-solution-cdiscount-starter


# Q&A from open-solution-talking-data
https://github.com/neptune-ml/open-solution-talking-data

# Q&A from open-solution-avito-demand-prediction
https://github.com/neptune-ml/open-solution-avito-demand-prediction


# Q&A from open-solution-toxic-comments